In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Read in the data
df = pd.read_csv('crsp_daily_cut.csv')

In [4]:
df.head()

,Unnamed: 0,permno,caldt,shrcd,excd,siccd,prc,ret,vol,shr
0,109545,10026,2023-01-05,11,3,2052,149.60,-0.010713,60472.0,19229.0
1,109546,10026,2023-01-06,11,3,2052,152.64,0.020321,57034.0,19229.0
2,109547,10026,2023-01-09,11,3,2052,149.61,-0.019851,63214.0,19229.0
3,109548,10026,2023-01-10,11,3,2052,149.87,0.001738,112759.0,19229.0
4,109549,10026,2023-01-11,11,3,2052,152.39,0.016815,57350.0,19229.0
